In [23]:
import numpy as np
import math
import glob
import pandas as pd
import re
import random

In [2]:
sent1 = "Julie loves India more than Linda loves India"
sent2 = "Jane likes India more than Julie loves India"

# sent1 = "Mahaprasad loves icecream more than he hates pizza"
# sent2 = "Mahaprasad is a good boy and obedient child"

## Find the cosine similarity


In [3]:
def find_cos_similarity(sent_a, sent_b):
    # make a list of unique words
    list_words = []
    for word in sent_a.split():
        if word not in list_words:
            list_words.append(word)
    for word in sent_b.split():
        if word not in list_words:
            list_words.append(word)
            
    # create 2 vectors for the words
    vector_a = []
    for word in list_words:
        vector_a.append(sent_a.count(word))
    vector_b = []
    for word in list_words:
        vector_b.append(sent_b.count(word))
    
    #find the dot product
    dot_product = 0

    for i in range(len(vector_a)):
        dot_product += vector_a[i] * vector_b[i]
        
    mod_a = 0
    sum = 0
    for i in vector_a:
        # print(i**2)
        sum += (i**2)
    mod_a = pow(sum, 0.5)
    
    mod_b = 0
    sum = 0
    for i in vector_b:
        sum += (i**2)
    mod_b = pow(sum, 0.5)
    mod_b

    cos_theta = dot_product/(mod_a * mod_b)
    
    return cos_theta

In [4]:
x = "I am a disco dancer and what are you"
y = " x"
find_cos_similarity(sent1, sent2)

0.8215838362577491

In [2]:
# extracting file names from folder
file_names = glob.glob('./data/English_text/*.txt')
len(file_names)

135

In [6]:
strings = []

for elem in range(len(file_names)):
    try:
        with open(file_names[elem], "r") as file:
            # print(elem)
            data = file.read()
            strings.append(data)
    except UnicodeDecodeError:
        continue

# with open(file_names[0], "r") as file:
#     data = file.read()

len(strings)


133

In [7]:
from chardet import detect

types = []
for elem in range(len(file_names)):
    with open(file_names[elem], 'rb') as file:
        data = file.read()
        encoding_type = detect(data)['encoding']
        
        if encoding_type not in types:
            types.append(detect(data)['encoding'])

types

['ascii', 'Windows-1252', 'UTF-8-SIG', 'ISO-8859-1']

## Creating a wrangle function to get data and give in the specific manner

In [3]:
def wrangle(file_names):
    strings = []
    for elem in range(len(file_names)):
        try:
            with open(file_names[elem], "r") as file:
                data = file.read()
                strings.append(data)
        except UnicodeDecodeError:
            continue
    
    # Remove symbols, numbers, and convert to lowercase
    cleaned_strings = [re.sub('[^A-Za-z]+', ' ', text).lower() for text in strings]
    
    print(f"Successfully read {len(strings)} files, removed symbols, numbers, and converted to lowercase.")
    return cleaned_strings


In [4]:
tokenized_corpus = wrangle(file_names)

Successfully read 133 files, removed symbols, numbers, and converted to lowercase.


In [5]:
def unique_words(list_strings):
    word_list = []
    for long_string in range(len(list_strings)):
        words = list_strings[long_string].split()
        
        for word in words:
            if word not in word_list:
                word_list.append(word)
    return word_list

In [24]:
vocabulary = unique_words(tokenized_corpus)

In [29]:
# def create_word_to_index(vocabulary):
    
#     word_to_index = {}

    
#     for index, word in enumerate(vocabulary):
        
#         word_to_index[word] = index

    
#     return word_to_index

word_to_index = {word: index for index, word in enumerate(vocabulary)}

In [14]:
word_to_index = create_word_to_index(vocabulary)
word_to_index['jaipur']

NameError: name 'create_word_to_index' is not defined

# convert each string to it's vector form

In [76]:
document_vectors = []

for document in tokenized_corpus:
    vector = [0] * len(vocabulary)
    for word in document:
        if word in word_to_index:
            vector[word_to_index[word]] +=1
    document_vectors.append(vector)


In [41]:
# print(sum(document_vectors[30]))
vocabulary.index('jaipur')
print(len(vocabulary))

24523


In [65]:

for i in range(len(document_vectors[0])):
    if document_vectors[0][i] == 8:
        print(i)


In [43]:
vocabulary[37]

'a'

In [99]:
tokenized_corpus[0].split().count('1929')

1

In [69]:
vocabulary[50]

'chand'

In [73]:
word_to_index['jaipur']

0

In [74]:
document_vectors[0][0]

0

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(tokenized_corpus)






In [9]:
X.shape

(133, 23258)

In [10]:
dense_array = X.toarray()

In [20]:
len(dense_array[2])

23258

# Finding euclidean distances

In [21]:
def euclidean_distance(vec1, vec2):
    if len(vec1) != len(vec2):
        raise ValueError("Vectors must have the same length")
    
    squared_distance = 0
    for i in range(len(vec1)):
        squared_distance += (vec1[i] - vec2[i]) ** 2
    
    return squared_distance ** 0.5

In [36]:
num_random_vectors = 3
random.seed(10)
random_indices = random.sample(range(X.shape[0]), num_random_vectors)
random_indices

[8, 109, 123]

In [37]:
selected_vectors = [X[i].toarray().ravel() for i in random_indices]
selected_vectors

[array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64)]

In [38]:
euclidean_distances = []
for i, document_vector in enumerate(X):
    distances = [euclidean_distance(document_vector.toarray().ravel(), selected_vector) for selected_vector in selected_vectors]
    euclidean_distances.append(distances)


In [39]:
euclidean_distances[0]

[115.2432210587677, 150.73818361649447, 128.2926342390708]

In [42]:
euclidean_distances = np.array(euclidean_distances)


group1 = []
group2 = []
group3 = []

# Define a small threshold for convergence
threshold = 1

# Initialize flags for convergence
converged1 = False
converged2 = False
converged3 = False

# Iterate until all groups converge
while not (converged1 and converged2 and converged3):
    # Reset group lists
    group1 = []
    group2 = []
    group3 = []
    
    # Iterate through the documents and their corresponding distances
    for distances in euclidean_distances:
        # Find the index (vector) with the smallest distance using NumPy argmin
        closest_vector = np.argmin(distances)
        
        # Assign the document to the corresponding group based on the closest vector
        if closest_vector == 0:
            group1.append(distances)
        elif closest_vector == 1:
            group2.append(distances)
        elif closest_vector == 2:
            group3.append(distances)
    
    # Calculate the means of the groups using NumPy
    mean1 = np.mean(group1, axis=0)
    mean2 = np.mean(group2, axis=0)
    mean3 = np.mean(group3, axis=0)
    
    # Check for convergence using NumPy and threshold
    converged1 = np.all(np.abs(mean1 - mean2) < threshold)
    converged2 = np.all(np.abs(mean2 - mean3) < threshold)
    converged3 = np.all(np.abs(mean3 - mean1) < threshold)

# Now, group1, group2, and group3 contain documents grouped based on the closest vector,
# and they have converged means.


KeyboardInterrupt: 